In [15]:
import numpy as np
import ome_zarr.io
import ome_zarr.writer
from dexp.datasets import ZDataset
import zarr
import napari

In [2]:
ds = ZDataset(r'/media/brandon/Data1/Brandon/fly_immune/Lightsheet_Z1/2022_02_24_uas-mcd8-gfp_r4-gal4_x_dipt_dtom2/ecoli_hs_gfp/larvae_4/crop.zarr', 'r')

├ Configured the number of threads used by BLOSC: 8
├ Opening existing Zarr storage: '/media/brandon/Data1/Brandon/fly_immune/Lightsheet_Z1/2022_02_24_uas-mcd8-gfp_r4-gal4_x_dipt_dtom2/ecoli_hs_gfp/larvae_4/crop.zarr' with read/write mode: 'r' and store type: 'None'
├ Opening as Directory store
├ Opening with mode: r
├ Exploring Zarr hierarchy...
├ Found channel: dpt-red
├ Found array: dpt-red
├ Found array: dpt-red_projection_0
├ Found array: dpt-red_projection_1
├ Found array: dpt-red_projection_2
├ Found channel: mem-green
├ Found array: mem-green
├ Found array: mem-green_projection_0
├ Found array: mem-green_projection_1
├ Found array: mem-green_projection_2


In [31]:
path_to_ome_zarr = r'/media/brandon/Data1/Brandon/fly_immune/Lightsheet_Z1/2022_02_24_uas-mcd8-gfp_r4-gal4_x_dipt_dtom2/ecoli_hs_gfp/larvae_4/mem.ome.zarr'
loc = ome_zarr.io.parse_url(path_to_ome_zarr, mode="w")  

# create a zarr root level group at the file path
group = zarr.group(loc.store)

    

In [32]:
# test with brute force numpy
ds_shape = ds.get_array('dpt-red').shape
ymax = 2000
ds_shape = (ds_shape[0], 1,) +  (ds_shape[1],) + (ymax,) + (ds_shape[3:])

big_arr = np.zeros(ds_shape)
channel_name = 'mem-green'
big_arr[0, 0] = ds.get_array(channel_name)[0, :, :2000, :]
# channel_names = ['dpt-red', 'mem-green']
# for c in range(big_arr.shape[1]):
#     big_arr[0,c] = ds.get_array(channel_names[c])[0, :, :2000, :]

In [33]:
scaler = ome_zarr.scale.Scaler()
ome_zarr.writer.write_image(big_arr, group=group, scaler=scaler)

In [40]:
viewer = napari.Viewer()
viewer.open(path_to_ome_zarr)


[<Image layer 'mem.ome' at 0x7f7992391ee0>]

In [41]:
viewer.layers[0].scale = [1, 1, 2, 0.304, 0.304]

/home/brandon/anaconda3/envs/ome_zarr/lib/python3.9/site-packages/napari/layers/image/image.py:676: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(


In [38]:
viewer.layers[0].scale

array([1.   , 1.   , 6.579, 1.   , 1.   ])

Based on Jordao's code, it looks like all napari-ome-zarr needs to read a pyramid file is a zarr with arrays named "0", "1", etc, for each of the downsampled images. His functions use  tiled, cupy-based pyramid computation on a slice by slice basis.